# LAB3 report
ใน lab นี้ ผู้ทำ lab จะต้องส่งข้อมูลจาก imu ที่อยู่บน Arduino โดยใช้ xicro_pkg เพื่อสื่อสารกับ ros2 โดยใช้ค่า imu ในการควบคุม serial manipulator

## ros2control parameter
ในการควบคุม serial manipulator เราจะใช้ velocity_controllers/JointGroupVelocityController ในการควบคุมความเร็วของแต่ละ joint โดย command interface คือ velocity และ state interface คือ position และ velocity เพื่อทำ feedback control

<img src="controller_setup.png">

และใส่ Gazebo plugin ของ ros2control ใน xacro

<img src="xacro_ros2control.png">

## IMU calibration
สร้าง node calibration_backend ในการ subscribe ค่าจาก /Imu_arduino และใช้ Action ในการเก็บข้อมูลหา covariance ของ sensor ซึ่งใช้ Multiexecutor ในการทำให้ subscriber และ Action callback สามารถใช้งานพร้อมกันได้ โดยการแบ่งให้อยู่คนละ callback group

<img src="imu_calibration_init.png">

โดยเราจะเก็บค่าที่ subscribe เข้ามาใส่ในตัวแปรและให้ Action ระหว่าง execute goal ดึงค่าไปเก็บ

<img src="imu_calibrate_sub.png">

โดยรหว่าง execute goal จะเก็บค่าที่ได้จาก imu เข้า list จากนั้นจะหา covariance และส่งกลับไป

<img src="imu_calibrate_result.png">

โครงสร้างของ Action

<img src="action_structure.png">

## trajectory_gen
เป็น node ที่เมื่อเริ่มทำงานจะส่ง goal ไปที่ IMU calibration เพื่อหา covaraince และส่งเป็น result กลับมา จากนั้นจะ publish covariance และค่า angular velocity, linear acceleration ไปที่ /imu/data_raw ของ node complementary_filter_node เพื่อทำการประมาณค่า orientation และจากนั้นจะ subscibe ค่าที่ถูกประมาณจาก /imu/data และนำค่าที่ได้แปลงให้อยู่ในรูป roll pitch yaw แบบ extrensic โดยใช้ scipy

<img src="trajec_init.png">

หลังจาก init node แล้วเราจะส่ง Goal ให้กับ IMU calibration เพื่อรัยค่า covariance

<img src="sendgoal.png">

จากนั้นนำ covariance ใส่ให้กับ msg ที่ได้จาก arduino ก่อนส่งไปประมาณค่าที่ complementary filter และรับค่า linear acceleration กลับมาเพื่อทำ joystick ควบคุมทิศทาง end effector

<img src="imu_filter.png">

## Xicro read IMU raw node
เราได้ใช้ Xicro ในการสร้างการสื่อสารระหว่าง ros2 และ arduino โดยค่าที่ส่งมาจะเป็นค่า orientation ในรูปแบบ unit quaternion, angular vocity และ linear acceleration โดยเราใช้ MadgwickAHRS ในการประมาณค่า orientation จากค่า angular vocity และ linear acceleration ใน arduino ก่อนส่งมา

<img src="arduino_xicro.png">

## Algorithm X
เราใช้ linear acceleration ที่ได้มาในการควบคุมให้ joint เคลื่อนที่ โดยในขั้นตอนแรกหุ่นยนต์จะนำ feedback จาก state interface ไปใช้ในการ set home เมื่อ set home เรียบร้อยจึงจะสามารถควบคุมได้ โดย linear acceleration จะกำหนดทิศทางที่ end effector จะเคลื่อนที่ โดยคำนวณ jacobian และหาความเร็วเชิงมุมของ joint
โดยหุ่นยนต์จะรอให้การ calibration เสร็จสิ้น จากนั้นหุ่นยนต์จะเคลื่อที่ไปตำแหน่งเริ่มต้นเพื่อให้หลุดจาก sigularity จากนั้นถึงจะใช้ IMU ควบคุมได้

<img src='end_effector_control.png'>

คำนวณ Rotation matrix จากตาราง DH parameter สำหรับ Jacobian

<img src='rotationfromDH.png'>

คำนวณ Jacobian

<img src='jacobianmatrix.png'>

## วิธีการทดลอง
พิมพ์คำสั่งนี้เพื่อเริ่มการอ่านค่า arduino<br>
ros2 run xicro_pkg xicro_node_read_raw_imu_ID_3_arduino.py<br>
จากนั้นเปิดหน้าต่าง terminal ใหม่แล้วพิมพ์คำสั่งนี้เพื่อเริ่ม simulation การควบคุม serial manipulator<br>
ros2 launch fra333_lab3_01 lab3_kinematics.launch.py

<img src="showcase.png">

โครงสร้างของ node ต่างๆ

<img src='node_system.jpeg'>